In [1]:
import pandas as pd
import numpy as np
import time

import xgboost as xgb
import joblib

In [2]:
data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.fillna(0, inplace=True)

NameError: name 'data' is not defined

In [7]:
data = data.sample(frac=1).reset_index(drop=True)
parts = np.array_split(data, parts_count)

In [2]:
train_data = pd.read_pickle('pd_data_train.pkl')
valid_data = pd.read_pickle('pd_data_valid.pkl')
test_data = pd.read_pickle('pd_data_test.pkl')

In [4]:
test_data.shape

(214200, 217)

In [5]:
train_data.to_csv('pd_data_train.csv', index=False, header=False)
valid_data.to_csv('pd_data_valid.csv', index=False, header=False)
test_data.to_csv('pd_data_test.csv', index=False, header=False)

In [4]:
def get_X_y(data):
#     data = data[data.date_block_num > 11]
    X = data.drop(['date_block_num', 'item_cnt_month', 'shop_id', 'item_id'], axis=1)
    y = data['item_cnt_month']
    return X, y

In [7]:
train_X, train_y = get_X_y(train_data)
valid_X, valid_y = get_X_y(valid_data)
test_X, test_y = get_X_y(test_data)

In [8]:
del train_data
del valid_data
del test_data

<h1>XGboost</h1>

In [23]:
from sklearn.model_selection import train_test_split

In [15]:
import pandas as pd

In [18]:
train_data = pd.read_pickle('D:/ml_data/pd_data_train.pkl')
valid_data = pd.read_pickle('D:/ml_data/pd_data_valid.pkl')
test_data = pd.read_pickle('D:/ml_data/pd_data_test.pkl')

In [19]:
def get_X_y(data):
#     data = data[data.date_block_num > 11]
    X = data.drop(['date_block_num', 'item_cnt_month', 'shop_id', 'item_id'], axis=1)
    y = data['item_cnt_month']
    return X, y

In [20]:
train_X, train_y = get_X_y(train_data)
valid_X, valid_y = get_X_y(valid_data)
test_X, test_y = get_X_y(test_data)

In [21]:
del train_data
del valid_data
del test_data

In [24]:
X_train, X_valid, y_train, y_valid = train_test_split(train_X, train_y, test_size=0.3, random_state=123)

In [25]:
del train_X
del train_y
del X_valid
del y_valid

In [26]:
import xgboost as xgb

In [27]:
train_data = xgb.DMatrix(X_train, label=y_train)

In [28]:
valid_data = xgb.DMatrix(valid_X, label=valid_y)

In [29]:
test_data = xgb.DMatrix(test_X, label=test_y)

In [30]:
del X_train
del valid_X
del test_X

In [31]:
import gc
gc.collect()

87

In [15]:
from sklearn.model_selection import ParameterGrid, ParameterSampler

param_test = {
    'eta ':  [0.01, 0.05, 0.15, 0.3],
    'subsample': [0.5, 0.7, 0.9], 
    'alpha': [0, 1e-1, 1, 2, 5],
    'lambda': [0, 1e-1, 1, 2, 5],
    'max_depth' : [3, 5, 7],
    'metric': ['rmse'],
    'num_threads': [4],
}

sampler = ParameterSampler(param_test, n_iter=25, random_state=123)

In [16]:
num_round = 100
early_stopping_rounds = 10

In [2]:
import os

In [3]:
trained_models_dir = 'trained_models_dir'
trained_models_dir = trained_models_dir + '/xgb_2'
if not os.path.exists(trained_models_dir):
    os.makedirs(trained_models_dir)

In [19]:
def get_inner_dirs(path):
    for file in os.listdir(path):
        if os.path.isdir(os.path.join(path, file)):
            yield file

In [20]:
def __create_model_directory(models_dir) -> str:
    inner_dirs = list(get_inner_dirs(models_dir))
    folder_name = 'gs_{}'.format(len(inner_dirs))

    model_dir = os.path.join(models_dir, folder_name)
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)

    return model_dir, folder_name

In [33]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def rmse(y_actual, y_predicted):
    return sqrt(mean_squared_error(y_actual, y_predicted))

In [22]:
def get_X_y(data):
    #     data = data[data.date_block_num > 11]
    X = data.drop(['date_block_num', 'item_cnt_month', 'shop_id', 'item_id'], axis=1)
    y = data['item_cnt_month']
    return X, y

In [4]:
import json
import pandas as pd

In [5]:
search_results = []
search_results_file = os.path.join(trained_models_dir, 'search_results.json')

In [25]:
for params in sampler:
        print('current parameters : {}'.format(params))

        model_dir, folder_name = __create_model_directory(trained_models_dir)
        print('model directory : {}'.format(model_dir))

        bst = xgb.train(params, train_data, num_round, [(train_data, 'train'), (valid_data, 'valid')], early_stopping_rounds=early_stopping_rounds)
        bst.save_model(os.path.join(model_dir, 'model_filtered.txt'))
        
        train_pred = bst.predict(train_data)
        valid_pred = bst.predict(valid_data)
        test_pred = bst.predict(test_data)

        mean_cv_scores = [
            rmse(y_train, train_pred),
            rmse(valid_y, valid_pred),
            rmse(test_y, test_pred),
        ]
        
        write_message = dict(
            params=params,
            scores=dict(
                train=mean_cv_scores[0],
                val=mean_cv_scores[1],
                test=mean_cv_scores[2],
            ),
            model_path=model_dir,
        )

        search_results.append(write_message)

        with open(search_results_file, 'w') as file:
            json.dump(search_results, file)
            print('overwrite model parameters file ({})'.format(search_results_file))

current parameters : {'subsample': 0.7, 'num_threads': 4, 'metric': 'rmse', 'max_depth': 3, 'lambda': 2, 'eta ': 0.05, 'alpha': 2}
model directory : trained_models_dir/xgb_2\gs_0
[02:01:01] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[0]	train-rmse:1.33481	valid-rmse:1.07524
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 10 rounds.
[02:01:08] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[1]	train-rmse:1.26345	valid-rmse:1.03061
[02:01:16] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[2]	train-rmse:1.22184	valid-rmse:1.00326
[02:01:23] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[3]	train-rmse:1.19752	valid-rmse:0.988499
[02:01:31] src/tree/updater_prune.cc:74: tree pruning end, 

[50]	train-rmse:1.09566	valid-rmse:0.93583
[02:07:11] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[51]	train-rmse:1.09524	valid-rmse:0.935864
[02:07:19] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[52]	train-rmse:1.09453	valid-rmse:0.935647
[02:07:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[53]	train-rmse:1.09406	valid-rmse:0.935335
[02:07:33] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[54]	train-rmse:1.09362	valid-rmse:0.934874
[02:07:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[55]	train-rmse:1.09348	valid-rmse:0.934689
[02:07:47] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[56]	train-rmse:1.0932	valid-rmse:0.934552
[02:07:54] src/tree/up

[26]	train-rmse:1.11132	valid-rmse:0.946186
[02:13:03] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[27]	train-rmse:1.10747	valid-rmse:0.943602
[02:13:10] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[28]	train-rmse:1.10608	valid-rmse:0.942185
[02:13:17] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[29]	train-rmse:1.10566	valid-rmse:0.941884
[02:13:23] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[30]	train-rmse:1.10486	valid-rmse:0.940717
[02:13:30] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[31]	train-rmse:1.10386	valid-rmse:0.940072
[02:13:37] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[32]	train-rmse:1.10299	valid-rmse:0.939021
[02:13:44] src/tree/

[79]	train-rmse:1.07795	valid-rmse:0.92867
[02:19:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[80]	train-rmse:1.0777	valid-rmse:0.928394
[02:19:16] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[81]	train-rmse:1.07745	valid-rmse:0.928449
[02:19:23] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[82]	train-rmse:1.07713	valid-rmse:0.928374
[02:19:30] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[83]	train-rmse:1.07664	valid-rmse:0.928384
[02:19:37] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[84]	train-rmse:1.07639	valid-rmse:0.928731
[02:19:43] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[85]	train-rmse:1.07626	valid-rmse:0.928979
[02:19:50] src/tree/up

[9]	train-rmse:1.0549	valid-rmse:0.922173
[02:29:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 222 extra nodes, 0 pruned nodes, max_depth=7
[10]	train-rmse:1.0518	valid-rmse:0.924519
[02:29:43] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 206 extra nodes, 0 pruned nodes, max_depth=7
[11]	train-rmse:1.04912	valid-rmse:0.9248
[02:30:00] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 178 extra nodes, 0 pruned nodes, max_depth=7
[12]	train-rmse:1.04696	valid-rmse:0.924585
[02:30:17] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 182 extra nodes, 0 pruned nodes, max_depth=7
[13]	train-rmse:1.04507	valid-rmse:0.925008
[02:30:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 218 extra nodes, 0 pruned nodes, max_depth=7
[14]	train-rmse:1.04165	valid-rmse:0.925056
[02:30:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 210 extra nodes, 0 pruned nodes, max_depth=7
[15]	train-rmse:1.03985	valid-rmse:0.925741
[02:31:07] src/tree

...

[33]	train-rmse:1.11171	valid-rmse:0.942172
[05:33:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[34]	train-rmse:1.11093	valid-rmse:0.941835
[05:33:12] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[35]	train-rmse:1.11008	valid-rmse:0.941117
[05:33:19] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[36]	train-rmse:1.10968	valid-rmse:0.94107
[05:33:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[37]	train-rmse:1.10907	valid-rmse:0.941597
[05:33:33] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[38]	train-rmse:1.1082	valid-rmse:0.941259
[05:33:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[39]	train-rmse:1.10692	valid-rmse:0.941254
[05:33:48] src/tree/up

[86]	train-rmse:1.08285	valid-rmse:0.934587
[05:39:27] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[87]	train-rmse:1.08274	valid-rmse:0.934497
Stopping. Best iteration:
[77]	train-rmse:1.08484	valid-rmse:0.933724

overwrite model parameters file (trained_models_dir/xgb_2\search_results.json)
current parameters : {'subsample': 0.9, 'num_threads': 4, 'metric': 'rmse', 'max_depth': 3, 'lambda': 0, 'eta ': 0.3, 'alpha': 0.1}
model directory : trained_models_dir/xgb_2\gs_22
[05:39:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[0]	train-rmse:1.33533	valid-rmse:1.0753
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 10 rounds.
[05:39:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[1]	train-rmse:1.26487	valid-rmse:1.03025
[05:39:47] src/tr

[48]	train-rmse:1.09308	valid-rmse:0.932368
[05:45:07] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[49]	train-rmse:1.09275	valid-rmse:0.932004
[05:45:14] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[50]	train-rmse:1.09211	valid-rmse:0.931529
[05:45:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[51]	train-rmse:1.09162	valid-rmse:0.931702
[05:45:28] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[52]	train-rmse:1.09107	valid-rmse:0.931743
[05:45:35] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[53]	train-rmse:1.09047	valid-rmse:0.931009
[05:45:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[54]	train-rmse:1.09003	valid-rmse:0.930931
[05:45:48] src/tree/

[9]	train-rmse:1.15195	valid-rmse:0.966997
[05:50:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10]	train-rmse:1.14966	valid-rmse:0.965524
[05:51:02] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[11]	train-rmse:1.14487	valid-rmse:0.962441
[05:51:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[12]	train-rmse:1.14214	valid-rmse:0.960823
[05:51:17] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[13]	train-rmse:1.13504	valid-rmse:0.956499
[05:51:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[14]	train-rmse:1.13262	valid-rmse:0.955766
[05:51:32] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[15]	train-rmse:1.12658	valid-rmse:0.953234
[05:51:39] src/tree/u

[12]	train-rmse:1.04689	valid-rmse:0.921089
[05:59:27] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 172 extra nodes, 0 pruned nodes, max_depth=7
[13]	train-rmse:1.04502	valid-rmse:0.921704
[05:59:45] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 234 extra nodes, 0 pruned nodes, max_depth=7
[14]	train-rmse:1.0413	valid-rmse:0.921768
[06:00:02] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 200 extra nodes, 0 pruned nodes, max_depth=7
[15]	train-rmse:1.0395	valid-rmse:0.921019
[06:00:19] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 160 extra nodes, 0 pruned nodes, max_depth=7
[16]	train-rmse:1.0383	valid-rmse:0.920378
[06:00:37] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 242 extra nodes, 0 pruned nodes, max_depth=7
[17]	train-rmse:1.03518	valid-rmse:0.920188
[06:00:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 222 extra nodes, 0 pruned nodes, max_depth=7
[18]	train-rmse:1.03323	valid-rmse:0.919399
[06:01:13] src/tr

In [6]:
with open(search_results_file, 'r') as file:
    search_results = json.load(file)

In [7]:
search_results

[{'params': {'subsample': 0.7,
   'num_threads': 4,
   'metric': 'rmse',
   'max_depth': 3,
   'lambda': 2,
   'eta ': 0.05,
   'alpha': 2},
  'scores': {'train': 1.0844269270103102,
   'val': 0.9381862036393841,
   'test': 0.866815745082543},
  'model_path': 'trained_models_dir/xgb_2\\gs_0'},
 {'params': {'subsample': 0.9,
   'num_threads': 4,
   'metric': 'rmse',
   'max_depth': 3,
   'lambda': 2,
   'eta ': 0.3,
   'alpha': 1},
  'scores': {'train': 1.0711352673475982,
   'val': 0.9279462712884965,
   'test': 0.8621689548136373},
  'model_path': 'trained_models_dir/xgb_2\\gs_1'},
 {'params': {'subsample': 0.5,
   'num_threads': 4,
   'metric': 'rmse',
   'max_depth': 7,
   'lambda': 0.1,
   'eta ': 0.3,
   'alpha': 2},
  'scores': {'train': 1.0397539634546498,
   'val': 0.9269927262737903,
   'test': 0.9031684706630548},
  'model_path': 'trained_models_dir/xgb_2\\gs_2'},
 {'params': {'subsample': 0.7,
   'num_threads': 4,
   'metric': 'rmse',
   'max_depth': 7,
   'lambda': 0,
   'e

In [8]:
valid_scores = [float(record['scores']['val']) for record in search_results]
min_index = valid_scores.index(min(valid_scores))

In [9]:
best_model = search_results[min_index]
best_model

{'params': {'subsample': 0.9,
  'num_threads': 4,
  'metric': 'rmse',
  'max_depth': 7,
  'lambda': 0,
  'eta ': 0.15,
  'alpha': 5},
 'scores': {'train': 0.9977327513134557,
  'val': 0.9097569780933839,
  'test': 0.9369717381203909},
 'model_path': 'trained_models_dir/xgb_2\\gs_13'}

In [12]:
bst = xgb.Booster({'nthread': 4})  # init model
bst.load_model(os.path.join(best_model['model_path'], 'model_filtered.txt'))  # load data

In [32]:
_train_data = pd.read_pickle('pd_data_train.pkl')
_valid_data = pd.read_pickle('pd_data_valid.pkl')
_test_data = pd.read_pickle('pd_data_test.pkl')

train_X, train_y = get_X_y(_train_data)
valid_X, valid_y = get_X_y(_valid_data)
test_X, test_y = get_X_y(_test_data)

del _train_data
del _valid_data
del _test_data

del train_X
del valid_X
del test_X

In [33]:
train_pred.shape

(3344082,)

In [34]:
# train_pred = bst.predict(xgb.DMatrix(train_X, label=train_y))
# valid_pred = bst.predict(xgb.DMatrix(valid_X, label=valid_y))
# test_pred = bst.predict(xgb.DMatrix(test_X, label=test_y))

train_pred = bst.predict(train_data)
valid_pred = bst.predict(valid_data)
test_pred = bst.predict(test_data)


mean_cv_scores = [
    rmse(y_train, train_pred),
    rmse(valid_y, valid_pred),
    rmse(test_y, test_pred),
]

In [35]:
mean_cv_scores

[0.9977326915733635, 0.9097569780933839, 0.9369717381203909]

In [36]:
import pickle

model_name = 'xgb'
with open('{}_train_pred.pickle'.format(model_name), 'wb') as f:
    pickle.dump(train_pred, f)

with open('{}_valid_pred.pickle'.format(model_name), 'wb') as f:
    pickle.dump(valid_pred, f)

with open('{}_test_pred.pickle'.format(model_name), 'wb') as f:
    pickle.dump(test_pred, f)

In [37]:
from module.data.read_data import *

test = test_file_processing().set_index('ID')
test.shape

(214200, 3)

In [42]:
import pickle

Y_test = bst.predict(test_data).clip(0, 20)

# print(test_X.shape)
print(Y_test.shape)

submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": Y_test
})
submission.to_csv('xgb_submission.csv', index=False)

# save predictions for an ensemble
# pickle.dump(Y_pred, open('xgb_train.pickle', 'wb'))
pickle.dump(Y_test, open('xgb_test.pickle', 'wb'))

(214200,)


In [43]:
submission.head()

,ID,item_cnt_month
0,0,0.682111
1,1,0.314219
2,2,1.232342
3,3,0.630036
4,4,4.781566


In [44]:
submission.min()

ID                0.0
item_cnt_month    0.0
dtype: float64

In [45]:
submission.max()

ID                214199.0
item_cnt_month        20.0
dtype: float64

In [46]:
plot_features(model, (10,14))

NameError: name 'plot_features' is not defined

In [ ]:
_sorted = {k: v for k, v in sorted(_fi.items(), key=lambda item: item[1], reverse=True)}
_sorted

In [ ]:
new_columns = [k for k, v in _sorted.items() if v > 4]
new_columns

In [26]:
from module.data.read_data import *

test = test_file_processing().set_index('ID')
test.shape

(214200, 3)

In [27]:
import pickle

Y_test = bst.predict(xg_test).clip(0, 20)

print(test.shape)
print(Y_test.shape)

submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": Y_test
})

submission.to_csv('xgb_submission.csv', index=False)
pickle.dump(Y_test, open('xgb_test.pickle', 'wb'))

(214200, 3)
(214200,)
